In [5]:
import sklearn.metrics as metrics
import matplotlib as plt
import numpy as np

import pandas as pd
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')
stores_train.head(10)

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528
5,998035295-979540094-54657,2016,BURGER KING STAVANGER,1.1.1.0,Hamburger restaurants,11030509,TORGET 7,58.970249,5.731728,BURGER KING,NaN,12.020
6,916757433-916772262-822439,2016,BURGER KING NARVIK,1.1.1.0,Hamburger restaurants,18050706,BOLAGSGATA 1,68.438025,17.422916,BURGER KING,Amfi Narvik,7.622
7,913158733-913203305-675470,2016,GRENSEN BURGERBAR,1.1.1.0,Hamburger restaurants,1010301,OLAV VS GATE 1,59.119175,11.389572,NaN,NaN,3.640
8,982174139-982239737-14482,2016,MCDONALD'S ÅSANE SENTER,1.1.1.0,Hamburger restaurants,12010915,ÅSANE SENTER 37,60.465118,5.324101,MCDONALDS,Åsane Storsenter,30.514
9,916217153-911834553-435311,2016,BERGBYS NOTODDEN,1.1.1.0,Hamburger restaurants,8070301,NESØYA,59.559079,9.249380,BERGBYS,NaN,6.339


In [6]:
stores_train.revenue.mean()

8.220683101329808

In [7]:
import sklearn.ensemble as ensemble


#testModel = ensemble.RandomForestRegression()

def rmsle(y_true, y_pred):
    return metrics.mean_squared_log_error(y_true, y_pred)**0.5


class ReallyBadRandomGuesser:
    """
    Model that fits a uniform distribution to the minimum and 
    maximum observed y-values in the training set.
    
    Args:
        random_seed (int): Seed for the random distribution used to 
            sample predictions.
    """
    
    def __init__(self, random_seed=None):
        self.random = np.random.RandomState(random_seed)
    
    def fit(self, X, y):
        # Store min/max values of train set y values 
        self.y_min = y.min()
        self.y_max = y.max()
    
    def predict(self, X):
        n = len(X)
        return self.random.uniform(self.y_min, self.y_max, size=n)

# Partition into X (not really used here) and y values 
X_train = stores_train.drop(columns=['revenue'])
y_train = stores_train.revenue 

# Create and fit a model 
model = ReallyBadRandomGuesser(random_seed=123)
model.fit(X_train, y_train)

# Generate predictions over the training set 
y_train_pred = model.predict(X_train)

print(f'Train set RMSLE: {rmsle(y_train, y_train_pred) :.4f}')

Train set RMSLE: 3.1491


In [11]:
import sklearn.ensemble as ensemble
import lightgbm as lgb

#newModel = ensemble.RandomForestRegressor()

def rmsle(y_true, y_pred):
    return metrics.mean_squared_log_error(y_true, y_pred)**0.5

def getX(stores):
    X = pd.DataFrame()
    X['h'] = stores.plaace_hierarchy_id.astype('category')
    return X

lgbtestModel = lgb.LGBMRegressor()
#stores_train.head(10)

x_train = getX(stores_train)
x_test = getX(stores_test)
y_train = stores_train.revenue


lgbtestModel.fit(x_train,y_train)
y_test_pred = lgbtestModel.predict(x_test)

#rmsle(y_train, y_test_pred)

#rmsle(y_test_pred, lgbtestModel.predict(x_train))
#Dette er en test om noe som helst funker


In [ ]:
# Predict on the test set 
X_test = stores_test  
y_test_pred = lgbtestModel.predict(X_test)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test['store_id']
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submissionFiles\\test1853.csv', index=False)
submission

In [12]:
#               Gamle metode
#y_test_pred 

#submission = pd.DataFrame()
#submission['id'] = stores_test['store_id']
#submission['predicted'] = y_test_pred